In [1]:
!pip install progressbar2
import progressbar

In [2]:
import pandas

In [14]:
cycle = 72
df_sensors = pandas.read_csv("data/cycle_{}.csv".format(cycle))
df_water = df_sensors[(df_sensors['sensor_type']=='FC28') & (df_sensors['value_type']=='watermark')]

In [15]:
df_photos = pandas.read_csv("data/photo_cycle_{}.csv".format(cycle))
df_photos['watermark'] = 0

In [16]:
df_photos.head()

,id,photo,created,seconds_from_cycle_start,watermark
0,267123,https://cress.space/media/photo/c72/1488132601...,2017-02-26T18:10:17.231219Z,602523,0
1,267119,https://cress.space/media/photo/c72/1488132301...,2017-02-26T18:05:16.970436Z,602223,0
2,267113,https://cress.space/media/photo/c72/1488132001...,2017-02-26T18:00:13.378223Z,601919,0
3,267109,https://cress.space/media/photo/c72/1488131701...,2017-02-26T17:55:12.428359Z,601618,0
4,267107,https://cress.space/media/photo/c72/1488131401...,2017-02-26T17:50:18.687203Z,601325,0


In [17]:
df_water.head()

,sensor_type,value_type,position,unit,value,created,seconds_from_cycle_start
1,FC28,watermark,inside,-,7506.0,2017-02-26T18:10:25.527352Z,602531
8,FC28,watermark,inside,-,7866.0,2017-02-26T18:05:24.046715Z,602230
15,FC28,watermark,inside,-,7911.0,2017-02-26T18:00:20.767143Z,601927
22,FC28,watermark,inside,-,7906.0,2017-02-26T17:55:21.099111Z,601627
29,FC28,watermark,inside,-,7890.0,2017-02-26T17:50:26.419153Z,601332


In [18]:
from bisect import bisect_left

def return_closest(myList, myNumber):
    pos = bisect_left(myList, myNumber)
    if pos == 0:
        return myList[0]
    if pos == len(myList):
        return myList[-1]
    before = myList[pos - 1]
    after = myList[pos]
    if after - myNumber < myNumber - before:
       return after
    else:
       return before

correlate

In [19]:
bar = progressbar.ProgressBar(max_value=len(df_photos))

water_list_seconds = sorted(df_water['seconds_from_cycle_start'].values)
for idx, photo_ds in bar(df_photos.iterrows()):
    a = return_closest(water_list_seconds, photo_ds['seconds_from_cycle_start'])
    df_photos.loc[idx, 'watermark'] = (df_water[df_water['seconds_from_cycle_start'] == a]['value']).values[0]
    df_photos.loc[idx, 'watermark_seconds_from_start'] = a

100% (2008 of 2008) |#####################| Elapsed Time: 0:00:03 Time: 0:00:03


In [20]:
df_photos[['watermark', 'watermark_seconds_from_start']] = df_photos[['watermark', 'watermark_seconds_from_start']].astype(int)

In [21]:
df_photos.head()

,id,photo,created,seconds_from_cycle_start,watermark,watermark_seconds_from_start
0,267123,https://cress.space/media/photo/c72/1488132601...,2017-02-26T18:10:17.231219Z,602523,7506,602531
1,267119,https://cress.space/media/photo/c72/1488132301...,2017-02-26T18:05:16.970436Z,602223,7866,602230
2,267113,https://cress.space/media/photo/c72/1488132001...,2017-02-26T18:00:13.378223Z,601919,7911,601927
3,267109,https://cress.space/media/photo/c72/1488131701...,2017-02-26T17:55:12.428359Z,601618,7906,601627
4,267107,https://cress.space/media/photo/c72/1488131401...,2017-02-26T17:50:18.687203Z,601325,7890,601332


In [22]:
df_photos.to_csv("data/photo_cycle_{}_enriched.csv".format(cycle))